# Benchmarking: numpy

Timing a plain numpy implementation of some basic mathematical operations.

## User input

In [1]:
import numpy as np

# storage
dtype = np.float64
storage_shape = (321, 321, 120)

# serialization
serialize = True
filename = "timings_daint.xlsx"
sheetname = "{} x {} x {}".format(*storage_shape)

## Data initialization

In [2]:
from copy import deepcopy

a = np.zeros(storage_shape, dtype=dtype)
a[...] = np.random.rand(*storage_shape)
a_dc = deepcopy(a)
b = np.zeros(storage_shape, dtype=dtype)
b[...] = np.random.rand(*storage_shape)
c = np.zeros(storage_shape, dtype=dtype)
c[...] = np.random.rand(*storage_shape)
d = np.zeros(storage_shape, dtype=dtype)

f = np.random.rand(1).item()

## Serialization setup

In [3]:
# install and import openpyxl
is_installed = !pip list 2> /dev/null | grep 'openpyxl'
if not is_installed:
    !pip install openpyxl==2.6
import openpyxl as xl

# install and import pandas
is_installed = !pip list 2> /dev/null | grep 'pandas'
if not is_installed:
    !pip install pandas
import pandas as pd

# install xlrd
is_installed = !pip list 2> /dev/null | grep 'xlrd'
if not is_installed:
    !pip install xlrd

    100% |████████████████████████████████| 112kB 635kB/s ta 0:00:01
You are using pip version 9.0.3, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
# dataframe's indices and column
index = [
    "copy", 
    "copychange", 
    "abs", 
    "iabs", 
    "add", 
    "iadd", 
    "sub", 
    "isub", 
    "mul", 
    "imul",
    "addsub",
    "iaddsub",
    "fma",
    "sts_rk2_0",
    "sts_rk3ws_0"
]
column = "numpy"
    
if serialize:
    # create the spreadsheet
    import os
    if not os.path.exists(filename):
        wb = xl.Workbook()
        wb.save(filename=filename)
    else:
        wb = xl.load_workbook(filename)
        
    # create an ExcelWriter object
    with pd.ExcelWriter(filename, engine="openpyxl") as writer:
        writer.book = wb
        # writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)
    
    if sheetname not in wb.sheetnames:
        # create an empty dataframe
        df = pd.DataFrame(data=[None,]*len(index), index=index, columns=[column,])
        df.to_excel(writer, sheet_name=sheetname)
    else:
        # load the dataframe
        df = pd.read_excel(writer, sheet_name=sheetname, index_col=0)
        
        # remove the sheet
        sheetid = wb.sheetnames.index(sheetname)
        wb.remove(wb.worksheets[sheetid])
else:
    # create the dataframe
    df = pd.DataFrame(data=[None,]*len(index), index=index, columns=[column,])

## Timing

In [5]:
# copy
out = %timeit -o a[...] = b[...]
df.at["copy", column] = 1000 * out.average

9.45 ms ± 4.77 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
# copychange
out = %timeit -o a = - b

40.7 ms ± 20.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
# copychange with assignment
out = %timeit -o a[...] = - b[...]
df.at["copychange", column] = 1000 * out.average

50.3 ms ± 189 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
# abs
out = %timeit -o b = np.abs(a)

40.7 ms ± 199 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
# abs with assignment
out = %timeit -o b[...] = np.abs(a)
df.at["abs", column] = 1000 * out.average

49.9 ms ± 28.3 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
# iabs with assignment
out = %timeit -o a[...] = np.abs(a)
df.at["iabs", column] = 1000 * out.average

50.1 ms ± 104 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
# add
out = %timeit -o c = a + b

45.4 ms ± 74.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
# add with assignment
out = %timeit -o c[...] = a[...] + b[...]
df.at["add", column] = 1000 * out.average

55.1 ms ± 66.4 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
# iadd with assignment
out = %timeit -o a[...] += b[...]
df.at["iadd", column] = 1000 * out.average

15.3 ms ± 10.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
# sub
out = %timeit -o c = a - b

45.5 ms ± 193 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
# sub with assignment
out = %timeit -o c[...] = a[...] - b[...]
df.at["sub", column] = 1000 * out.average

54.9 ms ± 219 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
# isub with assignment
out = %timeit -o a[...] -= b[...]
df.at["isub", column] = 1000 * out.average

15.5 ms ± 249 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
# mul
out = %timeit -o c = a * b

45.5 ms ± 133 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
# mul with assignment
out = %timeit -o c[...] = a[...] * b[...]
df.at["mul", column] = 1000 * out.average

55.1 ms ± 183 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
# imul with assignment
out = %timeit -o a[...] *= b[...]
df.at["imul", column] = 1000 * out.average

18.3 ms ± 1.44 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
# addsub
out = %timeit -o d = a + b - c

60.5 ms ± 54.2 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
# addsub with assignment
out = %timeit -o d[...] = a[...] + b[...] - c[...]
df.at["addsub", column] = 1000 * out.average

70.1 ms ± 113 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [22]:
# iaddsub with assignment
out = %timeit -o a[...] += b[...] - c[...]
df.at["iaddsub", column] = 1000 * out.average

60.7 ms ± 39.3 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
# fma
out = %timeit -o c = a + f * b

55.9 ms ± 189 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
# fma with assignment
out = %timeit -o c[...] = a[...] + f * b[...]
df.at["fma", column] = 1000 * out.average

65.1 ms ± 40.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
# sts_rk2_0
out = %timeit -o d = 0.5 * (a + b + f * c)

111 ms ± 163 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
# sts_rk2_0 with assignment
out = %timeit -o d[...] = 0.5 * (a[...] + b[...] + f * c[...])
df.at["sts_rk2_0", column] = 1000 * out.average

121 ms ± 309 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
# sts_rk3ws_0
out = %timeit -o d = (2.0 * a + b + f * c) / 3.0

137 ms ± 109 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [28]:
# sts_rk3ws_0 with assignment
out = %timeit -o d[...] = (2.0 * a[...] + b[...] + f * c[...]) / 3.0
df.at["sts_rk3ws_0", column] = 1000 * out.average

147 ms ± 235 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Serialization

In [29]:
if serialize:
    df.to_excel(writer, sheet_name=sheetname)
    writer.save()

In [30]:
df

,numpy
copy,9.44936
copychange,50.3371
abs,49.9262
iabs,50.1352
add,55.0943
iadd,15.2952
sub,54.9044
isub,15.5014
mul,55.0797
imul,18.3447
